In [15]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Json"
  // #r "nuget: Plotly.NET.ImageExport"

open Plotly.NET
// open Plotly.NET.ImageExport  

#load "Helpers.fs"
#load "Bestiary.fs"
#load "Compare.fs"
#load "DamageDistribution.fs"
#load "Library.fs"
#load "Transform.fs"

open PathfinderAnalysis.Library
open PathfinderAnalysis.Bestiary
open PathfinderAnalysis.Compare
open PathfinderAnalysis.Transform
open PathfinderAnalysis.Helpers

Installed Packages FSharp.Json, 0.4.1 Plotly.NET.Interactive, 5.0.0

In [8]:
let creatureLevelBump = creatureLevelBump

let titleFn level = 
  sprintf "Middle Save - Level %i - Spout Cantrip vs. Martial Bow Against Level %i Creatures" level (level+creatureLevelBump)

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel spout CreatureSave middleSave (casterDc true) bestiaryByLevel creatureLevelBump
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Spout (Middle)"
  };
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel telekineticProjectile PlayerAttack creatureAc (casterAttack true false) bestiaryByLevel creatureLevelBump
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Telekinetic Projectile"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel martialArbalest PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel creatureLevelBump
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Arbalest (1 shot, no PR)"
  };
  {
    AveragesByRollsByLevel = mapMerge (highMartialAttack true) creatureLevelBump 5 martialShortbow;
    Title = "Shortbow (2 shots, no PR)"
  }
|]
|> flatten
// |> generateCharts titleFn
|> generateLevelScaleChart (sprintf "Druid vs. Martial Accuracy - Creatures PL+%i" creatureLevelBump)
// |> Seq.iteri (fun i chart -> Chart.savePNG (path = (sprintf "spout-arbalest-%02i" i)) chart)
// |> ignore


In [14]:
let creatureLevelBump = 2

let titleFn level = 
  sprintf "Middle Save - Level %i - Caster Focus Spells vs. Fighter Weapons Against Level %i Creatures" level (level+creatureLevelBump)

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel tempestSurge CreatureSave middleSave (casterDc true) bestiaryByLevel creatureLevelBump
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Tempest Surge"
  };
  // { 
  //   AveragesByRollsByLevel = transformedResultsByRollByLevel fireRay PlayerAttack creatureAc (casterAttack true false) bestiaryByLevel creatureLevelBump
  //     |> Seq.map resultRollsToAverages
  //     |> Seq.toArray;
  //   Title = "Fire Ray";
  // };
  // {
  //   AveragesByRollsByLevel = transformedResultsByRollByLevel fighterArbalest PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel creatureLevelBump
  //     |> Seq.map resultRollsToAverages
  //     |> Seq.toArray;
  //   Title = "Arbalest (1 shot)"
  // };
  {
    AveragesByRollsByLevel = mapMerge (highFighterAttack true) creatureLevelBump 5 fighterLongbow
    Title = "Longbow (2 Strikes)"
  };
  {
    AveragesByRollsByLevel = mapMerge ((+) -2 << highFighterAttack true) creatureLevelBump 0 fighterLongbow
    Title = "Longbow (Double Shot)"
  }
  {
    AveragesByRollsByLevel = [
      transformedResultsByRollByLevel fighterLongbow PlayerAttack creatureAc ((+) -4 << highFighterAttack true) bestiaryByLevel creatureLevelBump |> Seq.map resultRollsToAverages;
      transformedResultsByRollByLevel fighterLongbow PlayerAttack creatureAc ((+) -4 << highFighterAttack true) bestiaryByLevel creatureLevelBump |> Seq.map resultRollsToAverages;
      transformedResultsByRollByLevel fighterLongbow PlayerAttack creatureAc ((+) -4 << highFighterAttack true) bestiaryByLevel creatureLevelBump |> Seq.map resultRollsToAverages;
    ]
    |> mergeNRollAverages
    Title = "Longbow (Triple Shot)"
  }
|]
|> flatten
// |> generateCharts titleFn 
|> generateLevelScaleChart (sprintf "Druid vs. Fighter - Creatures PL+%i" creatureLevelBump)


<!-- Plotly chart will be drawn inside this DIV -->

In [3]:
let monsterLevelDiff = 2

let titleFn level = 
  sprintf "Middle Save - Level %i - Caster Single-Target vs. Fighter Weapons Against Level %i Creatures" level (level+monsterLevelDiff)

let longswordAverage =
  transformedResultsByRollByLevel fighterLongsword PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
  |> Seq.map resultRollsToAverages

let shortswordAverage =
  transformedResultsByRollByLevel fighterShortsword PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
  |> Seq.map resultRollsToAverages

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel thunderstrike CreatureSave middleSave (casterDc true) bestiaryByLevel monsterLevelDiff
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Thunderstrike (Middle)"
  };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel longswordAverage shortswordAverage
      |> Seq.toArray;
    Title = "Double Slice (Swords)"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel (forceBarrage 3) PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
    |> Seq.map resultRollsToAverages
    |> Seq.toArray;
    Title = "3-action force barrage"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel (forceBarrage 2) PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
    |> Seq.map resultRollsToAverages
    |> Seq.toArray;
    Title = "2-action force barrage"
  }
|]
|> flatten
// |> generateCharts titleFn
|> generateLevelScaleChart (sprintf "Wizard vs. Fighter - Creatures PL+%i" monsterLevelDiff)

In [ ]:
open PathfinderAnalysis.DamageDistribution

let dicePool = [D12, 4; D6, 3; D4, 2]
let nBuckets = 20

let values = 
  rollDistributions 0 dicePool
  |> Seq.toList
  |> chunkRolls nBuckets
  |> chunksToAverages
  |> Seq.toList

let title = sprintf "Roll Sum of %A mapped onto a d%i" dicePool nBuckets

Chart.Line(
  x = List.map first values,
  y = List.map second values,
  ShowMarkers = true,
  MultiText = (List.map second values |> List.map (sprintf "%.0f")),
  TextPosition = StyleParam.TextPosition.TopCenter
)
|> Chart.withXAxisStyle(TitleText = "D20 Result")
|> Chart.withYAxisStyle(TitleText = "Estimated Roll Sum")
|> Chart.withTitle(title)



In [ ]:
open PathfinderAnalysis.DamageDistribution

let creatureLevelBump = 2

let titleFn level =
  sprintf "Level %i - Druid vs. Fighter Against Level %i Creatures (AC: %.1f DC: %.1f)" level (level + creatureLevelBump) (average creatureAc level) (average middleSave level + 10.0)

let fighterLongbowBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongbow pcLevel };
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongbow pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

// let fighterShortbowBucketsForLevel pcLevel =
//   [|
//     { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterShortbow pcLevel };
//     { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterShortbow pcLevel };
//   |]
//   |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let druidReflexBuckets pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = (fun c -> c.reflex + c.hasMagicBonus); DamageFunction = diceDamageTempestSurge pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let druidMiddleBuckets pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = middleSave; DamageFunction = diceDamageTempestSurge pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

[1..4]
|> List.map (fun level -> [
  "Tempest Surge (Reflex)", level, druidReflexBuckets level;
  "Longbow (2 Strikes)", level, fighterLongbowBucketsForLevel level;
  "Tempest Surge (Middle)", level, druidMiddleBuckets level;
  // "Shortbow (2 Strikes)", level, fighterShortbowBucketsForLevel level
])
|> Seq.collect (fun titledBucketsByLevel -> Seq.map (fun (title, level, bucket) -> chunksToFlatChartData title level bucket) titledBucketsByLevel)
|> generateCharts titleFn
// |> generateLevelScaleChart (sprintf "Reflex Save - Druid vs. Fighter Creatures PL+%i" creatureLevelBump)
